# Presentation du cas

L'objectif du cas est de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments absentes actuellement.

Les contraintes proposees sont les caracteristiques des batiments (taille et usage des bâtiments, mention de travaux récents, date de construction..)

Les donnees d'entrainement et de test utilisent des releves faits en 2015 et 2016.

Les bases de donnees sont composees de deux types de formats:
- JSON: les donnees descriptives qui renseignent les donnes brutes en format csv.
- CSV: les donnees brutes qui serviront a l'analyse exploratoire, au cleaning ainsi qu'a la modelisation finale.

La premiere partie consiste a selectionner et cleaner les donnees.

## Importations des librairies

In [176]:
# Standard libraries
import pandas as pd
import numpy as np
import sys
import os
import glob
import json
import warnings
import pickle
import string
warnings.filterwarnings('ignore')

# Graphic libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler



## Telechargements des donnees

In [2]:
folder_path=os.path.join("..","..","data","conso")
json_files=glob.glob(folder_path+"/*.json")
csv_files=glob.glob(folder_path+"/*.csv")
csv_1=pd.read_csv(csv_files[0])
csv_2=pd.read_csv(csv_files[1])
with open(json_files[0]) as json_data:
    json_1= json.load(json_data)
with open(json_files[1]) as json_data:
    json_2 = json.load(json_data)

## Exploration initiale des donnees

In [3]:
csv_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [4]:
csv_1.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3946027.0,12764.52930,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3242851.0,51450.81641,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,49526664.0,14938.00000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2768924.0,18112.13086,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5368607.0,88039.98438,8803998.0,False,NaN,Compliant,NaN,505.01,2.88


In [5]:
csv_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3340 entries, 0 to 3339
Data columns (total 47 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   OSEBuildingID                                                  3340 non-null   int64  
 1   DataYear                                                       3340 non-null   int64  
 2   BuildingType                                                   3340 non-null   object 
 3   PrimaryPropertyType                                            3340 non-null   object 
 4   PropertyName                                                   3340 non-null   object 
 5   TaxParcelIdentificationNumber                                  3338 non-null   object 
 6   Location                                                       3340 non-null   object 
 7   CouncilDistrictCode                                         

In [6]:
csv_2.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,...,GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes
0,1,2015,NonResidential,Hotel,MAYFLOWER PARK HOTEL,659000030,"{'latitude': '47.61219025', 'longitude': '-122...",7,DOWNTOWN,1927,...,2.64,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
1,2,2015,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,"{'latitude': '47.61310583', 'longitude': '-122...",7,DOWNTOWN,1996,...,2.38,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
2,3,2015,NonResidential,Hotel,WESTIN HOTEL,659000475,"{'latitude': '47.61334897', 'longitude': '-122...",7,DOWNTOWN,1969,...,1.92,Yes,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,18081
3,5,2015,NonResidential,Hotel,HOTEL MAX,659000640,"{'latitude': '47.61421585', 'longitude': '-122...",7,DOWNTOWN,1926,...,31.38,No,NaN,Compliant,High Outlier,NaN,56.0,NaN,31.0,18081
4,8,2015,NonResidential,Hotel,WARWICK SEATTLE HOTEL,659000970,"{'latitude': '47.6137544', 'longitude': '-122....",7,DOWNTOWN,1980,...,4.02,No,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,19576


In [7]:
csv_1.columns

Index(['OSEBuildingID', 'DataYear', 'BuildingType', 'PrimaryPropertyType',
       'PropertyName', 'Address', 'City', 'State', 'ZipCode',
       'TaxParcelIdentificationNumber', 'CouncilDistrictCode', 'Neighborhood',
       'Latitude', 'Longitude', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking',
       'PropertyGFABuilding(s)', 'ListOfAllPropertyUseTypes',
       'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
       'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA',
       'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA',
       'YearsENERGYSTARCertified', 'ENERGYSTARScore', 'SiteEUI(kBtu/sf)',
       'SiteEUIWN(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
       'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
       'Electricity(kWh)', 'Electricity(kBtu)', 'NaturalGas(therms)',
       'NaturalGas(kBtu)', 'DefaultData', 'Comments', 'ComplianceStatus',
       'Outlier

In [8]:
csv_2.columns

Index(['OSEBuildingID', 'DataYear', 'BuildingType', 'PrimaryPropertyType',
       'PropertyName', 'TaxParcelIdentificationNumber', 'Location',
       'CouncilDistrictCode', 'Neighborhood', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking',
       'PropertyGFABuilding(s)', 'ListOfAllPropertyUseTypes',
       'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
       'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA',
       'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA',
       'YearsENERGYSTARCertified', 'ENERGYSTARScore', 'SiteEUI(kBtu/sf)',
       'SiteEUIWN(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
       'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
       'Electricity(kWh)', 'Electricity(kBtu)', 'NaturalGas(therms)',
       'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)',
       'GHGEmissions(MetricTonsCO2e)', 'GHGEmissionsIntensity(kgCO2e/ft2)',
       'DefaultData', '

In [9]:
data=pd.concat([csv_1,csv_2]).reset_index()
data.head()

,index,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,...,Location,OtherFuelUse(kBtu),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),Comment,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes
0,0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data.isna().sum()

index                                                               0
OSEBuildingID                                                       0
DataYear                                                            0
BuildingType                                                        0
PrimaryPropertyType                                                 0
PropertyName                                                        0
Address                                                          3340
City                                                             3340
State                                                            3340
ZipCode                                                          3356
TaxParcelIdentificationNumber                                       2
CouncilDistrictCode                                                 0
Neighborhood                                                        0
Latitude                                                         3340
Longitude           

In [11]:
meta_data=pd.DataFrame(data.columns)
meta_data.columns=["column_names"]
meta_data["column_descriptions"]=np.nan
for i in range(0,len(json_1["columns"])):
    name=json_1["columns"][i]["name"]
    if "description" in json_1["columns"][i].keys():
        description=json_1["columns"][i]["description"]
        row=meta_data.column_names[meta_data.column_names == name].index[0]
        meta_data.iloc[row,1]=description
    
for i in range(0,len(json_2["columns"])):
    name=json_2["columns"][i]["name"]
    if "description" in json_2["columns"][i].keys():
        description=json_2["columns"][i]["description"]
        row=meta_data.column_names[meta_data.column_names == name].index[0]
        meta_data.iloc[row,1]=description
        
pd.set_option('display.max_colwidth', None)      
meta_data

,column_names,column_descriptions
0,index,NaN
1,OSEBuildingID,A unique identifier assigned to each property covered by the Seattle Benchmarking Ordinance for tracking and identification purposes.
2,DataYear,Calendar year (January-December) represented by each data record.
3,BuildingType,City of Seattle building type classification.
4,PrimaryPropertyType,"The primary use of a property (e.g. office, retail store). Primary use is defined as a function that accounts for more than 50% of a property. This is the Property Type - EPA Calculated field from Portfolio Manager."
5,PropertyName,Official or common property name.
6,Address,Property street address
7,City,Property city
8,State,Property state
9,ZipCode,Property zip


Les variables a predire sont:
- TotalGHGEmissions : les emissions de CO2
- SiteEnergyUse(kBtu) : consommation totale d'energie, toutes sources d'energie confondues

## Selection des colonnes

Les predictions devront etres basees sur les donnees declaratives des locaux. Nous gardons donc les colonnes suivantes:
- BuildingType
- PrimaryPropertyType
- YearBuilt
- NumberofBuildings
- NumberofFloors
- PropertyGFATotal
- PropertyGFAParking
- PropertyGFABuilding(s)
- LargestPropertyUseType
- LargestPropertyUseTypeGFA
- SiteEnergyUse(kBtu)
- TotalGHGEmissions

In [185]:
string.punctuation+"\n"

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n'

In [186]:
def remove_punctuations(text):
    for punctuation in string.punctuation+"\n":
        text = text.replace(punctuation, '')
    return text

columns=["BuildingType","PrimaryPropertyType","YearBuilt","NumberofBuildings",
                      "NumberofFloors","PropertyGFATotal","PropertyGFAParking",
                      "PropertyGFABuilding(s)","LargestPropertyUseType","LargestPropertyUseTypeGFA",
                      "SiteEnergyUse(kBtu)","TotalGHGEmissions"]
data_subset=data[columns]
categories=data_subset.select_dtypes(include="object")
categories=categories.applymap(lambda x: remove_punctuations(str(x).lower()))
dummies=pd.get_dummies(categories)
data_subset.drop(categories.columns,axis=1,inplace=True)
data_subset=pd.concat([data_subset,dummies],axis=1)

In [187]:
def data_prep(data,train=True,knn=None,mm=None):

    
    energy_data=data.drop(["TotalGHGEmissions"],axis=1)
    energy_data = energy_data[energy_data['SiteEnergyUse(kBtu)'].notna()]
    
    ghg_data=data.drop(["SiteEnergyUse(kBtu)"],axis=1)
    ghg_data = ghg_data[ghg_data['TotalGHGEmissions'].notna()]
        
    
    if train is True:
        energy_data_y=pd.DataFrame(energy_data["SiteEnergyUse(kBtu)"]).reset_index(drop=True)
        energy_data_x=energy_data.drop(["SiteEnergyUse(kBtu)"],axis=1).reset_index(drop=True)
        
        ghg_data_y=pd.DataFrame(ghg_data["TotalGHGEmissions"]).reset_index(drop=True)
        ghg_data_x=ghg_data.drop(["TotalGHGEmissions"],axis=1).reset_index(drop=True)
        
    else:
        energy_data_y=pd.DataFrame(energy_data["SiteEnergyUse(kBtu)"]).reset_index(drop=True)
        ghg_data_y=pd.DataFrame(ghg_data["TotalGHGEmissions"]).reset_index(drop=True)

        
        energy_data_x=energy_data.drop(["SiteEnergyUse(kBtu)"],axis=1).reset_index(drop=True)
        ghg_data_x=ghg_data.drop(["TotalGHGEmissions"],axis=1).reset_index(drop=True)
    
    if train is True:
        knn=KNNImputer()
        knn.fit_transform(pd.concat([energy_data_x,ghg_data_x],axis=0))
        
        energy_data_x=pd.DataFrame(knn.transform(energy_data_x),columns=energy_data_x.columns)
        ghg_data_x=pd.DataFrame(knn.transform(ghg_data_x),columns=ghg_data_x.columns)
            
    else:
        energy_data_x=pd.DataFrame(knn.transform(energy_data_x),columns=energy_data_x.columns)
        ghg_data_x=pd.DataFrame(knn.transform(ghg_data_x),columns=ghg_data_x.columns)
    
    if train is True:
        mm=MinMaxScaler()
        mm.fit_transform(pd.concat([energy_data_x,ghg_data_x],axis=0))
       
        energy_data_x=pd.DataFrame(mm.transform(energy_data_x),columns=energy_data_x.columns)
        ghg_data_x=pd.DataFrame(mm.transform(ghg_data_x),columns=ghg_data_x.columns)
        
    else:
        energy_data_x=pd.DataFrame(mm.transform(energy_data_x),columns=energy_data_x.columns)
        ghg_data_x=pd.DataFrame(mm.transform(ghg_data_x),columns=ghg_data_x.columns)        
    
    
    if train is True:
        return [energy_data_x,energy_data_y],[ghg_data_x,ghg_data_y],[knn,mm]
    else:
        return [energy_data_x,energy_data_y],[ghg_data_x,ghg_data_y]


In [188]:
train_data,test_data,_,_=train_test_split(data_subset,data_subset,train_size=0.8,shuffle=True,random_state=123)

In [189]:
[train_energy_data_x,train_energy_data_y],[train_ghg_data_x,train_ghg_data_y],[knn,mm]=data_prep(train_data,train=True)
[test_energy_data_x,test_energy_data_y],[test_ghg_data_x,test_ghg_data_y]=data_prep(test_data,train=False,knn=knn,mm=mm)

In [190]:
[col for col in train_energy_data_x.columns if 'PrimaryProperty' in col]

['PrimaryPropertyType_collegeuniversity',
 'PrimaryPropertyType_distribution center',
 'PrimaryPropertyType_highrise multifamily',
 'PrimaryPropertyType_hospital',
 'PrimaryPropertyType_hotel',
 'PrimaryPropertyType_k12 school',
 'PrimaryPropertyType_laboratory',
 'PrimaryPropertyType_large office',
 'PrimaryPropertyType_lowrise multifamily',
 'PrimaryPropertyType_medical office',
 'PrimaryPropertyType_midrise multifamily',
 'PrimaryPropertyType_mixed use property',
 'PrimaryPropertyType_nonrefrigerated warehouse',
 'PrimaryPropertyType_office',
 'PrimaryPropertyType_other',
 'PrimaryPropertyType_refrigerated warehouse',
 'PrimaryPropertyType_residence hall',
 'PrimaryPropertyType_residence halldormitory',
 'PrimaryPropertyType_restaurant',
 'PrimaryPropertyType_retail store',
 'PrimaryPropertyType_selfstorage facility',
 'PrimaryPropertyType_senior care community',
 'PrimaryPropertyType_small and midsized office',
 'PrimaryPropertyType_spsdistrict k12',
 'PrimaryPropertyType_supermark

## Save data

In [191]:
with open("train_data.pkl","wb") as f:
    pickle.dump([[train_energy_data_x,train_energy_data_y],[train_ghg_data_x,train_ghg_data_y],[knn,mm]],f)
with open("test_data.pkl","wb") as f:
    pickle.dump([[test_energy_data_x,test_energy_data_y],[test_ghg_data_x,test_ghg_data_y]],f)